In [25]:
from utils import *

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

np.set_printoptions(formatter={'float': '{: 0.2f}'.format})

In [26]:
# Bunny point cloud
# path_to_bunny = "D:\\Skole\\Semester 10\\Prosjektoppgave\\Data\\bunny\\reconstruction\\bun_zipper.ply"
path_to_bunny = "D:\\Runar\Documents\\1_Studier\\Semester_10\\Prosjektoppgave\\22v-TPK4560-Prosjektoppgave-Pose_estimation\\Data\\bunny" +\
    "\\reconstruction\\bun_zipper_res3.ply"
    # "\\data\\bun000.ply"
bunny_ply = o3d.io.read_point_cloud(path_to_bunny)
bunny = np.asarray(bunny_ply.points)
bunny_true = np.copy(bunny)
n = bunny.shape[0]

In [27]:
# Correct rotation
rot = np.array([np.pi/3,
              np.pi/3,
              np.pi/3
            ])
# Noise and outliers
# noise_1 = np.array([np.pi/3 + np.random.normal(0,0.05),
#                     np.pi/3 + np.random.normal(0,0.05),
#                     np.pi/3 + np.random.normal(0,0.05)
#                     ])
outlier = np.array([np.pi * np.random.uniform(-1,1),
                    np.pi * np.random.uniform(-1,1),
                    np.pi * np.random.uniform(-1,1)
                    ])
      

In [28]:
bunny = bunny @ expso3(rot)

In [29]:
# Generating translated cloud with inaccuracies
for i, p in enumerate(bunny):
    if i % 5 != 0:
        bunny[i] = expso3(outlier) @ p 
        # + np.random.uniform(-.1,.2, 3)
        outlier = np.array([np.pi*np.random.uniform(-1,1),
                    np.pi*np.random.uniform(-1,1),
                    np.pi*np.random.uniform(-1,1)
                    ])
        continue

In [30]:
# GNC Initialization
# Initial rotation guess
R0 = np.identity(3)

# Initial loss function data
r = np.zeros(n)
rtest = np.zeros(n)
for i in range(n):
    r[i] = np.linalg.norm(bunny_true[i] - R0 @ bunny[i])
    rtest[i] = np.linalg.norm(expso3(rot) @ bunny_true[i] - R0 @ bunny_true[i])
r0_max = np.max(r)

# gnc
eps = 0.011 
mu_update_factor = 1.4
max_iterations = 1000
w = np.ones(n)
mu = eps**2 / (2*r0_max**2 - eps**2)

In [31]:
last_iter = []
R_iter = [np.sum(rtest)]
iterations = 0
for i in range(max_iterations):
    iterations += 1
    last_iter.append(np.sum(w))
    # Weighted Procrustes
    H = bunny.T @ np.diag(w) @ bunny_true
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ np.diag([1,1,np.linalg.det( Vt.T @ U.T)]) @ U.T

    # Loss function
    for j in range(n):
        r[j] = np.linalg.norm(bunny_true[j] - R @ bunny[j])
        w[j] = w_from_r(r[j], eps, mu)
        rtest[j] = np.linalg.norm(expso3(rot) @ bunny_true[j] - R @ bunny_true[j])


    R_iter.append(np.sum(rtest))

    mu = mu_update_factor * mu

    if i >= 5:
        if np.sum(w) == last_iter[i]:
            break

In [32]:
print(R)
print()
print(expso3(rot))
print()
print(expso3(rot) - R)

[[ 0.17 -0.15  0.97]
 [ 0.97  0.17 -0.15]
 [-0.15  0.97  0.17]]

[[ 0.17 -0.15  0.97]
 [ 0.97  0.17 -0.15]
 [-0.15  0.97  0.17]]

[[ 0.00 -0.00 -0.00]
 [ 0.00 -0.00 -0.00]
 [ 0.00 -0.00  0.00]]


In [33]:
inliers = []
outliers = []

for i, n in enumerate(w):
    if w[i] == 1.0:
        inliers.append(n)
    else:
        outliers.append(n)

percentage = (1-len(inliers)/(len(outliers)+len(inliers)))*100

print("Inliers:\t", len(inliers), "\nOutliers:\t", len(outliers),\
    "\nPercentage:\t",percentage, "%\n\nIterations:\t", iterations)

Inliers:	 383 
Outliers:	 1506 
Percentage:	 79.72472207517205 %

Iterations:	 33


In [34]:
geometry_list = []

In [35]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(bunny)
# o3d.visualization.draw_geometries([pcd], width=1600, height=900)
# geometry_list.append(pcd)

In [36]:
color1 = np.array(([1.0, 0.0, 0.0]))
color2 = np.array(([0.0, 1.0, 0.0]))
color3 = np.array(([0.0, 0.0, 1.0]))
pcdtest1 = o3d.geometry.PointCloud()
pcdtest2 = o3d.geometry.PointCloud()
pcdtest3 = o3d.geometry.PointCloud()

In [37]:
bunnytest = bunny_true.copy()
test1 = bunnytest @ expso3(rot)
pcdtest1.points = o3d.utility.Vector3dVector(test1)
test2 = bunnytest @ R + np.ones((bunnytest.shape[0],3))*0.001
pcdtest2.points = o3d.utility.Vector3dVector(test2)
test3 = np.zeros((bunnytest.shape[0],3))
for i, x in enumerate(bunnytest):
    test3[i,:] = w[i] * x
pcdtest3.points = o3d.utility.Vector3dVector(test3)
# testfinal = np.concatenate((bunnytest, test1, test2, test3), axis=0)

In [38]:
pcdtest1.paint_uniform_color(color1)
pcdtest2.paint_uniform_color(color2)
pcdtest3.paint_uniform_color(color3)

geometry_list.append(pcdtest1)
geometry_list.append(pcdtest2)
geometry_list.append(pcdtest3)

In [39]:
# pcd1 = o3d.geometry.PointCloud()
# pcd1.points = o3d.utility.Vector3dVector(testfinal)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1, origin=[0, 0, 0])
geometry_list.append(mesh_frame)
o3d.visualization.draw_geometries(geometry_list, width=1600, height=900)

In [40]:
print(last_iter)

[1889.0, 6.497721917703617, 16.52427927904113, 78.83019570191124, 381.6495955972682, 381.9684745157424, 382.24448969445336, 382.47124030977534, 382.6410533372545, 382.75708772843245, 382.81817029136846, 382.8261145994473, 382.7837831821054, 382.7174929828551, 382.6874285521641, 382.7218065947562, 382.762973801862, 382.7891203020668, 382.80618461674743, 382.87262037198997, 383.01283623891857, 383.20910624467297, 383.0544569495343, 382.958522951075, 382.8656438664168, 382.841711763111, 382.9319591031475, 383.03362967402586, 383.14501431778035, 383.2388430110384, 383.15016646394633, 383.0256483894748, 383.0]


In [41]:
for i, n in enumerate(R_iter):
    print(n)

280.49858174111057
117.62845923658374
44.002237043771615
8.51948733619449
0.8050459571973033
0.09927975201481033
0.10069906925129676
0.10427829872798916
0.09996644231255847
0.08954188453365136
0.07406533124345877
0.0524403301416323
0.036708095406764604
0.022734443401460948
0.015197355861211575
0.014589464099039391
0.01654072158189056
0.018094886087449354
0.024328327114213514
0.031700776039771346
0.03116868377351018
0.0446502341931868
0.058067700488354884
0.05763332648414762
0.06342236512668224
0.06856985879280329
0.07876799543178761
0.09265845360009857
0.1125144056131329
0.14171413312428435
0.17397431117941353
0.1868137184678246
0.20491869250366915
0.20865771957574536
